## Solutions

### Problem 1
* Task description / introduction to SIR model: see [this notebook](SIRmodel.ipynb)
* We use [`odeint`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html) to solve the system of ODEs.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

In [ ]:
def F(y, t, N, c, w):
    "return the derivatives defining the differential equations of the SIR model, y = (S, I, R)"
    S, I, R = y
    dS = -c*S/N*I
    dI =  c*S/N*I -w*I
    dR =           w*I
    return dS, dI, dR

In [ ]:
# solve system of ODEs
def SolveSIRAndPlot(F, y0, ts, args):
    # solve ODE
    S, I, R = odeint(F, y0, ts, args).T
    # plot result
    plt.plot(ts, S, label = "susceptible")
    plt.plot(ts, I, label = "infected")
    plt.plot(ts, R, label = "recovered")
    plt.xlabel("time [days]")
    plt.ylabel("individuals [1]")
    plt.legend()
    plt.show()

In [ ]:
# time steps [assume days as time unit]
def TimeSteps(t_min, t_max, dt):
    return np.arange(t_min, t_max+dt, dt)

In [ ]:
# initial conditions and constants
y0 = (999, 1, 0) # (note: we use floats although individual counts are integers)
N  = sum(y0)
c  = 0.10
w  = 0.05

In [ ]:
print(f"R0 = {c/w} (basic reproduction number)")
if c/w > N/y0[0]: print("Outbreak")

In [ ]:
ts = TimeSteps(t_min = 0, t_max = 500, dt = 1)

In [ ]:
SolveSIRAndPlot(F, y0, ts, args = (N, c, w))

To get a feeling for the impact of the model parameters on the time evolution, we can use [``ipywidgets``](https://ipywidgets.readthedocs.io/)  in jupyter notebooks, which allow us to add interactive sliders to change the model parameters and call a function whenever the slider has been moved:

In [ ]:
from ipywidgets import interact
interact(lambda c: c**2, c=(0, 0.5, 0.05));

In [ ]:
%matplotlib inline
from ipywidgets import interact
interact(lambda c, w, t_max: 
             SolveSIRAndPlot(F, y0, TimeSteps(0, t_max, 5), 
                             args = (N, c, w)), 
         c=(0, 0.5, 0.05),
         w=(0, 0.5, 0.05),
         t_max=(100, 500, 100),
         continuous_update=False);

We can observe that the number of infected individuals starts decreasing once $S < N/R_0$.

This can also be derived from ${\frac {dI}{dt}}=c{\frac {SI}{N}}-wI = I (c \frac SN - w)$: 
${\frac {dI}{dt}}$ is negative iff $c \frac SN < w \Leftrightarrow S < \frac{wN}c = N/R_0$.

### Problem 2

This is an open-ended project task. Here, we will just look at some basics to get you started on your own investigations.

In [ ]:
### same code as in task
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from requests_cache import CachedSession # if you get an error, install this with: pip install requests_cache

In [ ]:
session = CachedSession('../data/cache_plot_corona_cases', backend='sqlite', expire_after = 86400)

def SaveURL(url, path):
  with open(path, "w") as outf:
    outf.write(GetFromURL(url))
    
def GetFromURL(url):
  response = session.get(url)
  print(f"Loaded {url} from cache: {response.from_cache}")
  if response.status_code != 200:
    print(f"Request failed with code {response.status_code}")
    return None
  else:
    return response.text

def LoadDataset(url, local_path, date_column, cases_column):
  SaveURL(url, local_path)
  df = pd.read_csv(local_path, parse_dates = [date_column])\
         .set_index(date_column)
  print("Last data point:")
  print(df.tail(1)[cases_column])
  return df

# RKI, documentation: https://github.com/robert-koch-institut/SARS-CoV-2-Nowcasting_und_-R-Schaetzung/#readme
dfr = LoadDataset("https://raw.githubusercontent.com/robert-koch-institut/SARS-CoV-2-Nowcasting_und_-R-Schaetzung/main/Nowcast_R_aktuell.csv", 
                  "/tmp/Nowcast_R_aktuell.csv",
                  "Datum",
                  "PS_COVID_Faelle")
# JHU, documentation: https://github.com/owid/covid-19-data/tree/master/public/data#readme
dfj = LoadDataset("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/jhu/new_cases.csv", 
                  "/tmp/new_cases.csv",
                  "date",
                  "Germany")

Now we can work with the data.

First, look at the data in tabular form:

In [ ]:
dfr.info()

In [ ]:
dfr.head()

In [ ]:
dfr.describe()

Next, it's always a good idea to plot a dataset to see whether it makes sense intuitively:

In [ ]:
def Plot_Curve(df, colname, label):
  l1 = plt.plot(df[colname], label = label)
  # mark last point with a star
  last = df.tail(1)
  plt.plot(last.index, last[colname], "*", color = l1[0].get_color())

In [ ]:
# the following two lines increase the size of the following plots
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 8] 
# plot RKI data
Plot_Curve(dfr, "PS_COVID_Faelle", "Nowcast")
# plot JHU data
Plot_Curve(dfj, "Germany", "JHU")
# style
plt.tick_params(right = True, labelright = True)
plt.xlabel("Date")
plt.ylabel("New cases / day (Germany)")
plt.legend()
plt.tight_layout()
###
plt.xlim(pd.Timestamp('2022-01')); # show from beginning of 2022
plt.plot(df[df["Germany"].isna()].fillna(0), "*"); # show nans (e.g. one point in 2022-08)

We observe:
* JHU has very strong fluctuations
* most recent days are missing in Nowcast

### Data clean-up

Create a shorthand for the JHU numbers for Germany:

In [ ]:
df = dfj[["Germany"]]

First issue with the data:  Sometimes the number of new cases per day are negative.

----
*It seems that the issue tackled in the following is no longer present -- maybe they have changed this, as it says, "In rare cases where our source for confirmed cases & deaths reports a negative daily change due to a data correction, we set the corresponding metric (new_cases or new_deaths) to NA.". (That's why we plotted the N/A above.)*

Reason given in README: 
> Note: the number of cases or deaths reported by any institution—including JHU, the WHO, the ECDC and others—on a given day does not necessarily represent the actual number on that date. This is because of the long reporting chain that exists between a new case/death and its inclusion in statistics. This also means that negative values in cases and deaths can sometimes appear when a country corrects historical data, because it had previously overestimated the number of cases/deaths. Alternatively, large changes can sometimes (although rarely) be made to a country's entire time series if JHU decides (and has access to the necessary data) to correct values retrospectively.

In [ ]:
df.describe() # "min" would be negative

Fortunately, pandas provides a handy solution:

In [ ]:
df.clip(lower = 0).describe()

What is the problem with this?

In [ ]:
df.sum() - df.clip(lower = 0).sum()

We're artificially reducing the sum. But wait... that's today's number...

In [ ]:
df.index[df["Germany"] < 0]

It seems only the last value is wrong, so probably not too much a concern.

----

Second issue with the data: Day-to-day fluctuations. Unlikely to be real.

Simplest solution: rolling average with [`pandas.DataFrame.rolling`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html). 
Uses [`pandas.DataFrame.assign`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.assign.html) to add a column to the dataframe.

In [ ]:
df = df.assign(rolling = df["Germany"].rolling("7d", center = True).mean())

In [ ]:
df.tail(20)

In [ ]:
# plot
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 8]
plt.plot(df["Germany"], "b", alpha = 0.4)
plt.plot(df["rolling"], "b");

Another idea: study tendency over weekdays

[pandas.DatetimeIndex](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html) has useful attributes, e.g., the week day.

In [ ]:
df = df.assign(weekday = df.index.weekday)
df

Reminder: there is [`loc` and `iloc`](https://stackoverflow.com/a/31593712/143931) to access rows and columns in pandas `DataFrame`s.

In [ ]:
df.loc["01.11.2020":"30.04.2021"].plot.scatter("weekday", "Germany", alpha = 0.2);

(Hard to see anything here as numbers vary a lot even when selecting a subset.)

In [ ]:
df = df.assign(avg = df["Germany"] / df["rolling"])

In [ ]:
df.tail(20)

In [ ]:
df.plot.scatter("weekday", "avg", alpha = 0.2);

We see a trend -- however, be careful with interpretation as the delay in reporting numbers is unclear. Also, we saw that the numbers for the last weeks are zero on weekends and holidays, whereas there are non-zero value in the above plot for all days, so clearly there are non-zero numbers for weekends at earlier times, too.

Let's quickly check that by selecting rows where the daily new cases are zero:

In [ ]:
df[df["Germany"] == 0][["Germany", "weekday"]]

Indeed, it seems for most weekends we do have data.